In [40]:
import yfinance as yf
import numpy as np
import pandas as pd
import jax.numpy as jnp

import sys
sys.path.append(r"C:/Users/olav2/OneDrive - Norwegian School of Economics/Skrivebord/Master/src")

from bs_module import BSModel


In [15]:
ticker = yf.Ticker("SPY")

spot = ticker.history(period="1d")["Close"].iloc[-1]
print(f"SPX spot: {spot:.2f}")


SPX spot: 637.10


In [24]:
expiries = ticker.options
expiry = expiries[6]      
print("Expiry valgt:", expiry)

Expiry valgt: 2025-08-15


In [25]:
chain = ticker.option_chain(expiry)
calls = chain.calls
puts  = chain.puts

In [27]:
atm_calls = calls.iloc[(calls["strike"] - spot).abs().argsort()[:1]]
K = float(atm_calls["strike"].iloc[0])
market_price = float(atm_calls["lastPrice"].iloc[0])
T = (pd.to_datetime(expiry) - pd.Timestamp.today()).days / 252

print(f"ATM strike: {K}, markedspris (call): {market_price:.2f}, T={T:.3f}")

ATM strike: 635.0, markedspris (call): 9.21, T=0.071


In [36]:
r = 0.0425   
q = 0.0    
model = BSModel(r=r, q=q)

# Modellpris
model_price = model.price(spot, K, T, sigma=0.2, option="call")
print(f"Modellpris (sigma=20%): {model_price:.4f}")

Modellpris (sigma=20%): 15.6486


In [37]:
# Cell 5: Finn implied vol
iv = model.implied_vol(
    market_price=market_price,
    S=spot,
    K=K,
    T=T,
    option="call",
    sigma0=0.2,
    tol=1e-6,
    maxiter=50
)
print(f"Implied vol: {iv:.4%}")

Implied vol: 10.3596%


In [41]:
nearest = calls.iloc[(calls["strike"]-spot).abs().argsort()[:5]]
Ks = nearest["strike"].values
mkt_prices = nearest["lastPrice"].values
Ts = np.full_like(Ks, T)

Ss = jnp.array([spot]*len(Ks))
Ks_j = jnp.array(Ks)
Ts_j = jnp.array(Ts)
sigmas_guess = jnp.full_like(Ks_j, 0.2)

# Vektorisert prising
model_prices = model.price_vec(Ss, Ks_j, Ts_j, sigmas_guess, 'call')
print("Markedspriser:", mkt_prices)
print("Modellpriser:", np.array(model_prices))

# Vektorisert implied vol (loop internt)
ivs = model.implied_vol(mkt_prices, Ss, Ks_j, Ts_j, option='call')
print("Implied vols:", np.array(ivs))

ValueError: vmap in_axes must be an int, None, or a tuple of entries corresponding to the positional arguments passed to the function, but got len(in_axes)=6, len(args)=5